# Ilusión de Mayoría en redes *scale-free* — Notebook
Este cuaderno replica el script standalone, adaptado a notebook.

**Qué hace:**
- Genera redes *scale-free* (Zipf + configuration model).
- Ajusta asortatividad por grado `r_kk` mediante rewiring dirigido.
- Asigna atributos con correlación objetivo `rho_kx` por bisección.
- Calcula la fracción de **ilusión de mayoría** (nodos con >50% vecinos activos).
- Intenta correr en paralelo y **guarda CSV + PNG** en `run/N{N}_{aammdd-hhmmss}/`.
- Muestra el gráfico *inline* y también lo guarda a archivo.

> En Windows/Jupyter, si `multiprocessing.Pool` falla por tema de *pickling*, el notebook usa **hilos** o ejecución secuencial como fallback.


In [1]:
# === Importaciones y configuración de Matplotlib (sin LaTeX) ===
import os, time, math
from pathlib import Path
import numpy as np
import networkx as nx
from scipy.stats import pearsonr

import matplotlib as mpl
import matplotlib.pyplot as plt

# Evitar uso de LaTeX y usar fuentes seguras (Windows friendly)
mpl.rcParams['text.usetex'] = False
mpl.rcParams['mathtext.fontset'] = 'dejavusans'
mpl.rcParams['font.family'] = 'DejaVu Sans'
mpl.rcParams['axes.unicode_minus'] = False

def timestamp():
    return time.strftime("%Y-%m-%d %H:%M:%S")

def ts_compacto():
    # aammdd-hhmmss (año 2 dígitos)
    return time.strftime("%y%m%d-%H%M%S")

def seed_everything(seed: int):
    np.random.seed(seed)

In [2]:
# === Funciones del experimento ===
import copy

def generate_scale_free_network(N, alpha, seed=None):
    """Genera una red scale-free por Zipf(alpha) + configuration model (grafo simple)."""
    #print(f'[{timestamp()}][alpha={alpha}] Generando red scale-free N={N}')
    start = time.time()
    rng = np.random.default_rng(seed)
    k_min, k_max = 1, int(np.sqrt(N))
    degrees = []
    while len(degrees) < N:
        k = rng.zipf(alpha)
        if k_min <= k <= k_max:
            degrees.append(int(k))
    if sum(degrees) % 2 == 1:
        degrees[0] += 1
    G = nx.configuration_model(degrees, seed=seed)
    G = nx.Graph(G)
    G.remove_edges_from(nx.selfloop_edges(G))
    #print(f'[{timestamp()}][alpha={alpha}] Red creada en {time.time()-start:.2f}s | N={G.number_of_nodes()} M={G.number_of_edges()}')
    return G

def strong_rewire(G, target_rkk, max_iter_factor=10, tol=0.01, alpha=None):
    """Double-edge swap dirigido para acercar r_kk a target_rkk (preserva grados)."""
    iter_count = 0
    current_rkk = nx.degree_assortativity_coefficient(G)
    #print(f'[{timestamp()}][alpha={alpha}] Iniciando rewiring rkk={current_rkk:.4f} target={target_rkk}')
    start = time.time()
    M = G.number_of_edges()
    max_iter = max_iter_factor * max(1, M)
    edges = list(G.edges())
    if M < 2:
        #print(f'[{timestamp()}][alpha={alpha}] Red con M<2, se omite rewiring.')
        return G
    while abs(current_rkk - target_rkk) > tol and iter_count < max_iter:
        i1, i2 = np.random.choice(M, 2, replace=False)
        e1, e2 = edges[i1], edges[i2]
        if len({e1[0], e1[1], e2[0], e2[1]}) != 4:
            iter_count += 1
            continue
        u, v = e1; x, y = e2
        if u == y or x == v or G.has_edge(u, y) or G.has_edge(x, v):
            iter_count += 1
            continue
        G_temp = G.copy()
        G_temp.remove_edge(u, v); G_temp.remove_edge(x, y)
        G_temp.add_edge(u, y);    G_temp.add_edge(x, v)
        new_rkk = nx.degree_assortativity_coefficient(G_temp)
        if abs(new_rkk - target_rkk) < abs(current_rkk - target_rkk):
            G = G_temp
            current_rkk = new_rkk
            edges = list(G.edges()); M = len(edges)
        iter_count += 1
    #print(f'[{timestamp()}][alpha={alpha}] Rewiring final rkk={current_rkk:.4f} en {iter_count} iters y {time.time()-start:.2f}s')
    return G

def bisection_attribute_assignment(G, px1, target_rho_kx, max_iter=50, tol=0.01, alpha=None, rkk=None, seed=None):
    """Asigna atributos binarios buscando rho(k,x)≈target_rho_kx con bisección en un sesgo por grado."""
    rng = np.random.default_rng(seed)
    #print(f'[{timestamp()}][alpha={alpha}, rkk={rkk}] Ajustando atributos para rho_kx={target_rho_kx}')
    start = time.time()
    n = G.number_of_nodes()
    degrees = np.array([deg for _, deg in G.degree()], dtype=float)
    if degrees.max() == degrees.min():
        degrees_norm = np.zeros_like(degrees)
    else:
        degrees_norm = (degrees - degrees.min()) / (degrees.max() - degrees.min())
    low, high = 0.0, 1.0
    best_attrs = None
    best_corr_diff = float('inf')
    n_active = int(round(px1 * n))
    for _ in range(max_iter):
        theta = 0.5 * (low + high)
        prob_active = (1 - theta) * np.ones(n) / n + theta * (degrees_norm + 1e-12)
        prob_active /= prob_active.sum()
        attrs = np.zeros(n, dtype=int)
        idx = rng.choice(n, n_active, p=prob_active, replace=False)
        attrs[idx] = 1
        corr = pearsonr(degrees, attrs)[0]
        corr_diff = abs(corr - target_rho_kx)
        if corr_diff < best_corr_diff:
            best_corr_diff = corr_diff
            best_attrs = attrs.copy()
        if corr > target_rho_kx:
            high = theta
        else:
            low = theta
        if best_corr_diff < tol:
            break
    #print(f'[{timestamp()}][alpha={alpha}, rkk={rkk}] Asignación terminada en {time.time()-start:.2f}s con corr_diff={best_corr_diff:.4f}')
    return best_attrs

def majority_illusion_fraction(G, attributes, alpha=None, rkk=None, rho_kx=None):
    """Fracción de nodos con >50% de vecinos activos."""
    #print(f'[{timestamp()}][alpha={alpha}, rkk={rkk}, rho_kx={rho_kx}] Calculando fracción espejismo')
    start = time.time()
    n = G.number_of_nodes()
    adj = nx.adjacency_matrix(G, nodelist=range(n))
    attrs = np.array(attributes, dtype=int).reshape(-1, 1)
    counts = adj @ attrs
    degrees = np.array([d for _, d in G.degree()], dtype=float).reshape(-1, 1)
    illusion = (counts > (degrees / 2.0)).astype(int)
    illusion_frac = float(illusion.sum() / n)
    #print(f'[{timestamp()}][alpha={alpha}, rkk={rkk}, rho_kx={rho_kx}] Fracción: {illusion_frac:.3f}, tiempo {time.time()-start:.2f}s')
    return illusion_frac

def experiment(args):
    alpha, rkk_target, rho_kx, G_base, px1, max_rewire_factor, tol, seed = args
    try:
        np.random.seed(seed)
        G = copy.deepcopy(G_base)
        #print(f"\n[{timestamp()}][alpha={alpha}] Inicio experimento rkk={rkk_target}, rho_kx={rho_kx}")
        G = strong_rewire(G, rkk_target, max_iter_factor=max_rewire_factor, tol=tol, alpha=alpha)
        attrs = bisection_attribute_assignment(G, px1, rho_kx, alpha=alpha, rkk=rkk_target, seed=seed + 777)
        frac = majority_illusion_fraction(G, attrs, alpha=alpha, rkk=rkk_target, rho_kx=rho_kx)
        #print(f'[{timestamp()}][alpha={alpha}] Resultado rkk={rkk_target}, rho_kx={rho_kx} --> frac={frac:.3f}')
        return (alpha, rkk_target, rho_kx, frac)
    except Exception as e:
        print(f'[{timestamp()}][alpha={alpha}] ERROR en experimento ({rkk_target}, {rho_kx}): {e}')
        return (alpha, rkk_target, rho_kx, float('nan'))

## Parámetros del experimento

In [3]:
# Parámetros
N = 100
px1 = 0.05
max_rewire_factor = 3#5
tol = 0.035
seed_global = 12021974
processes = 16
rho_min, rho_max, rho_steps = 0.0, 0.6, 4 #0.0, 0.6, 10
def parse_rkk_map():
    return {
        2.1: [-0.35, -0.25, -0.15, -0.05],
        2.4: [-0.20, -0.10,  0.20],
        3.1: [-0.15, -0.05, 0.30],
    }
'''
def parse_rkk_map():
    return {
        2.1: [-0.35, -0.25, -0.15, -0.05],
        2.4: [-0.20, -0.10,  0.00,  0.10, 0.20],
        3.1: [-0.15, -0.05,  0.00,  0.30],
    }
'''
params = parse_rkk_map()
rho_kxs = np.linspace(rho_min, rho_max, rho_steps)

## Ejecución y guardado de resultados

In [ ]:
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool

t0 = time.time()

base_dir = Path("run").resolve()
sub_dir = base_dir / f"N{N}_{ts_compacto()}"
sub_dir.mkdir(parents=True, exist_ok=True)
print(f"[{timestamp()}] Salidas se guardarán en: {sub_dir}")
print(f"[{timestamp()}] Parámetros: N={N}, px1={px1}, rho∈[{rho_min},{rho_max}], steps={rho_steps}, procesos={processes}")
seed_everything(seed_global)

#print(f'[{timestamp()}] Generando redes base...')
base_networks = {}
for alpha in sorted(params.keys()):
    base_networks[alpha] = generate_scale_free_network(N, alpha, seed=42 + int(alpha * 10))

param_list = []
base_seed = seed_global
for alpha, rkk_list in params.items():
    G_base = base_networks[alpha]
    for rkk in rkk_list:
        for rho_kx in rho_kxs:
            param_list.append((alpha, rkk, float(rho_kx), G_base, px1, max_rewire_factor, tol, base_seed))
            base_seed += 1

print(f'[{timestamp()}] Total de corridas: {len(param_list)}')

results = None
try:
    if processes and processes > 1:
        with Pool(processes=processes) as pool:
            results = pool.map(experiment, param_list)
    else:
        results = list(map(experiment, param_list))
except Exception as e:
    print(f'[{timestamp()}] Aviso: Pool falló ({e}). Intento con hilos...')
    try:
        with ThreadPool(processes=min(processes or 4, os.cpu_count() or 4)) as pool:
            results = pool.map(experiment, param_list)
    except Exception as e2:
        print(f'[{timestamp()}] Aviso: hilos fallaron ({e2}). Ejecución secuencial...')
        results = list(map(experiment, param_list))

print(f'[{timestamp()}] Experimentos completados.')

results_dict = {}
for alpha, rkk, rho_kx, frac in results:
    results_dict.setdefault(alpha, {}).setdefault(rkk, []).append((rho_kx, frac))

csv_path = sub_dir / "resultados_majority_illusion.csv"
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    import csv
    writer = csv.writer(f)
    writer.writerow(["alpha", "r_kk", "rho_kx", "frac_majority_illusion"])
    for alpha in sorted(results_dict.keys()):
        for rkk in sorted(results_dict[alpha].keys()):
            for rho_kx, frac in sorted(results_dict[alpha][rkk], key=lambda x: x[0]):
                writer.writerow([alpha, rkk, rho_kx, frac])
print(f"[{timestamp()}] Resultados guardados en: {csv_path}")

# Plot
fig, axes = plt.subplots(1, len(sorted(params.keys())), figsize=(18, 5), sharey=True)
if len(sorted(params.keys())) == 1:
    axes = [axes]
for idx, alpha in enumerate(sorted(params.keys())):
    ax = axes[idx]
    for rkk in params[alpha]:
        data = sorted(results_dict[alpha][rkk], key=lambda x: x[0])
        x = [d[0] for d in data]
        y = [d[1] for d in data]
        ax.plot(x, y, marker='o', label=f'r_kk={rkk}')
    ax.set_title(f'α={alpha}')
    ax.set_xlabel('Correlación grado-atributo ρₖₓ')
    ax.grid(True, alpha=0.3, linestyle=':')
    if idx == 0:
        ax.set_ylabel('Fracción con mayoría de vecinos activos')
    ax.legend(loc="best", fontsize=8)

fig.suptitle('Ilusión de Mayoría en Redes Scale-Free por α', y=1.02, fontsize=14)
try:
    fig.tight_layout()
except Exception as e:
    print(f"[{timestamp()}] Aviso: tight_layout falló ({e}). Uso subplots_adjust como respaldo.")
    plt.subplots_adjust(wspace=0.3, hspace=0.3, left=0.07, right=0.98, top=0.88, bottom=0.12)

png_path = sub_dir / "majority_illusion_plot.png"
fig.savefig(png_path, dpi=150, bbox_inches="tight")
print(f"[{timestamp()}] Figura guardada en: {png_path}")

elapsed = time.time() - t0
print(f"[{timestamp()}] Tiempo total de ejecución: {elapsed:.2f}s (~{elapsed/60:.2f} min)")

plt.show()

[2025-11-07 01:37:54] Salidas se guardarán en: C:\Users\dcaceres\Documents\Magister\Trimestre 2\CS Redes\Tarea 2\python\run\N100_251107-013754
[2025-11-07 01:37:54] Parámetros: N=100, px1=0.05, rho∈[0.0,0.6], steps=4, procesos=16
[2025-11-07 01:37:54] Total de corridas: 40
